In [ ]:
##################################################################################################################
#
# GuitarML Proteus Colab Capture Instructions
#
#  Step 1. Click "Runtime" in the Colab top menu bar, then "Change Runtime Type"
#          and select "GPU" and save. You have a limited number of consecutive GPU hours with
#          the free version of Colab, but this will reset in a day or so. 
#
#  Step 2. Upload your recorded output from your device by dragging and dropping the "out.wav"
#          file to the file browser in the left hand window in Colab. Ensure that this file is named
#          "out.wav".
#
#  Step 3. Run the SETUP ENVIRONMENT section by clicking the run arrow on the top left of the box.
#          Should take a few minutes. Scroll down to next step when finished.
#
#  Step 4. Choose one (and only one) of the next 3 options as applicable to your captured device. This
#          will begin the model training process. It should take about 10 minutes with GPU runtime selected.
#          The training may finish before reaching 300 epochs, this is normal.
#          
#  Step 5. Run this section to generate a Proteus compatible model file. Download from the 
#          left hand file browser by right clicking the "newProteusModel.json" filename in the left hand menu and 
#          selecting "Download". May need to refresh the filebrowser by clicking the refresh folder icon.
#
#  Step 6. Optionally, generate a plot of the signals to visually see how close the model is 
#          to the actual device. 
#
#  Note: Recommended to "Disconnect and delete runtime" from the "Runtime" menu and "Reconnect" to reset the environment
#     when training additional models. Make sure to download your trained model before disconnecting.
#
#  Note: To continue to refine the model, you may run the same Step 4 (a,b,c) after changing the "-eps" field to 
#     the number of epochs to continue training.
#     For example, in Step 4c., change "-eps 300" to "-eps 100" to run an additional 100 epochs. 
#     Then re-run Step 5 to generate a new Proteus compatible model. 
#
##################################################################################################################


In [ ]:
# STEP 3 : SETUP ENVIRONMENT
# Run this block to setup environment. May take a few minutes.

!git clone https://github.com/GuitarML/Automated-GuitarAmpModelling.git
%cd Automated-GuitarAmpModelling/
!git checkout proteus-capture
!git submodule update --init --recursive

In [ ]:
# STEP 4a. Run this to start capture of CLEAN AMP, EDGE OF BREAKUP AMP, COMPRESSOR
model="amp_clean"
!python prep_wav.py $model -s Data/Proteus_Capture.wav ../out.wav --normalize true
!python dist_model_recnet.py -l "RNN3-"$model -eps 300

In [ ]:
# STEP 4b. Run this to start capture of MEDIUM to HIGH GAIN AMP
model="amp_gain"
!python prep_wav.py $model -s Data/Proteus_Capture.wav ../out.wav --normalize true
!python dist_model_recnet.py -l "RNN3-"$model -eps 300

In [ ]:
# STEP 4c.  Run this to start capture of OVERDRIVE, DISTORTION, or BOOST PEDAL
model="pedal"
!python prep_wav.py $model -s Data/Proteus_Capture.wav ../out.wav --normalize true
!python dist_model_recnet.py -l "RNN3-"$model -eps 300

In [3]:
# STEP 5. After Above Capture Process is Completed, Run this to generate a Proteus compatible model file.
#         The model file will be located in the top level folder with the name "newProteusModel.json"
#         May need to refresh the left hand file browser by clicking the refresh folder icon.
#         Right click the file and choose download, then rename as appropriate.

%cp "Results/"$model"-RNN3-"$model"/model_best.json" ../newProteusModel.json

In [ ]:
# STEP 6. (Optional) After above Capture Process is completed, Run this to generate data graphs of the device model
#         The PNG image will be located in the top level folder with the name "detail_signal_comparison_e2s...".
#         May need to refresh the left hand file browser by clicking the refresh folder icon.

# Double click the PNG file to view the graphs. These graphs show approximately 10 milliseconds of audio signal. 
# The top plot shows the input signal. The middle plot shows the target device signal vs the signal 
#      predicted by the new model. The goal is for these signals to be as close as possible.

!python plot.py $model
%cp "Results/"$model"-RNN3-"$model"/"$model"_Detail"* ../

In [ ]:
## TROUBLESHOOTING NOTES ##

# 1. Training will go through 300 epochs, beginning with a pre-trained starting point model.
#    The number of epochs can be adjusted by changing the "-eps" value in the dist_model_recnet.py 
#    line on step 4 (a,b,c). You can run the same training step again to refine the model, and stop at any time. 
#
# 2. This catpure method is limited to non-time based effects (no Delay, Reverb, Flange, Phaser, etc.) 
#      It is intended for Amplifiers, Distortion/Overdrive/Boost and Compression(results on compressors vary)
# 